In [1]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [2]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [3]:
# Load Dataset
df = pd.read_csv('../../df.csv')

In [4]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [5]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [6]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [7]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks. 
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [8]:
# Select the length of the df ; For this file we need 16 months
# We use the first 12 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 16 months needed)
df = df.sort_index().loc['2021-09-01':'2022-12-30']

In [9]:
#Set the DATETIME for fecha 
DATETIME = df.index.values[::29]

In [10]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day 
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [11]:
# Select individual stocks and remove demographics
aapl = df.iloc[::29  ,:74]
f    = df.iloc[10::29,:74]
nvda = df.iloc[22::29,:74]
tsla = df.iloc[25::29,:74]
wmt  = df.iloc[27::29,:74]

In [12]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(aapl['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(aapl['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_1 = Sequential()
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_1.add(Dense(100, activation = 'relu', input_dim = aapl.iloc[:,1:-1].shape[1]))
model_1.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_1.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_1.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

aapl_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = aapl.iloc[i:train_rows + i, 1:] 
    test   = aapl.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_1.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_1.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    aapl_result = pd.concat([aapl_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
24/24 [==============================] - 1s 3ms/step - loss: 24664.0293
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 21038.6113
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 10321.7305
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 1300.8148
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 351.5083
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 293.4162
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 270.3608
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 238.1070
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 240.3885
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 223.0637
Epoch 11/200
24/24 [==============================] - 0s 3ms/step - loss: 220.2645
Epoch 12/200
24/24 [==============================] - 0s 3ms/step - loss: 211.5277
Epoch 

Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 200.6065
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 189.8744
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 199.6280
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 195.9340
Epoch 5/200
24/24 [==============================] - 0s 2ms/step - loss: 189.3381
Epoch 6/200
24/24 [==============================] - 0s 2ms/step - loss: 188.8648
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 193.7339
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 191.8472
Epoch 9/200
24/24 [==============================] - 0s 2ms/step - loss: 194.6998
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 193.9799
Epoch 10: early stopping
Count Down: 6
Epoch 1/200
24/24 [==============================] - 0s 3ms/step - loss: 191.1030
Epoch 2/200
24/24 [==============================] - 0s 3m

In [13]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(f['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(f['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_2 = Sequential()
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_2.add(Dense(100, activation = 'relu', input_dim = f.iloc[:,1:-1].shape[1]))
model_2.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_2.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_2.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

f_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = f.iloc[i:train_rows + i, 1:] 
    test   = f.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_2.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_2.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    f_result = pd.concat([f_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
24/24 [==============================] - 1s 3ms/step - loss: 205.8733
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 30.1800
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 6.6650
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 4.2120
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 3.2419
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 2.7239
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 2.4073
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 2.2462
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 2.2087
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 2.1974
Epoch 11/200
24/24 [==============================] - 0s 3ms/step - loss: 2.0322
Epoch 12/200
24/24 [==============================] - 0s 3ms/step - loss: 2.0834
Epoch 13/200
24/24 [==============

In [14]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(nvda['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(nvda['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_3 = Sequential()
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_3.add(Dense(100, activation = 'relu', input_dim = nvda.iloc[:,1:-1].shape[1]))
model_3.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_3.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_3.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

nvda_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = nvda.iloc[i:train_rows + i, 1:] 
    test   = nvda.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_3.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_3.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    nvda_result = pd.concat([nvda_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
24/24 [==============================] - 1s 3ms/step - loss: 51671.2305
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 44322.6328
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 23657.3965
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 5267.4180
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 2169.2485
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 1571.8821
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 1175.2200
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 925.5298
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 751.2509
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 676.0985
Epoch 11/200
24/24 [==============================] - 0s 3ms/step - loss: 571.5431
Epoch 12/200
24/24 [==============================] - 0s 3ms/step - loss: 533.8693
Epo

24/24 [==============================] - 0s 3ms/step - loss: 222.1722
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 233.3818
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 225.0327
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 227.6660
Epoch 7/200
24/24 [==============================] - 0s 2ms/step - loss: 222.7270
Epoch 7: early stopping
Count Down: 1
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 221.8109
Epoch 2/200
24/24 [==============================] - 0s 2ms/step - loss: 224.9133
Epoch 3/200
24/24 [==============================] - 0s 2ms/step - loss: 224.9399
Epoch 4/200
24/24 [==============================] - 0s 2ms/step - loss: 223.4498
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 226.3165
Epoch 5: early stopping
Count Down: 0


In [15]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(tsla['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(tsla['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_4 = Sequential()
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_4.add(Dense(100, activation = 'relu', input_dim = tsla.iloc[:,1:-1].shape[1]))
model_4.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_4.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_4.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

tsla_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = tsla.iloc[i:train_rows + i, 1:] 
    test   = tsla.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_4.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_4.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    tsla_result = pd.concat([tsla_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
24/24 [==============================] - 1s 3ms/step - loss: 88785.9609
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 80307.6406
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 52402.3516
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 16006.4688
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 3014.8835
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 2095.2983
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 1789.9763
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 1514.2744
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 1340.0591
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 1174.2385
Epoch 11/200
24/24 [==============================] - 0s 3ms/step - loss: 1101.0065
Epoch 12/200
24/24 [==============================] - 0s 3ms/step - loss: 1021.06

Epoch 13: early stopping
Count Down: 9
Epoch 1/200
24/24 [==============================] - 0s 2ms/step - loss: 707.2108
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 647.3872
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 645.2888
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 660.5736
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 654.8464
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 640.2663
Epoch 7/200
24/24 [==============================] - 0s 3ms/step - loss: 673.9123
Epoch 8/200
24/24 [==============================] - 0s 3ms/step - loss: 647.7388
Epoch 9/200
24/24 [==============================] - 0s 3ms/step - loss: 650.3174
Epoch 10/200
24/24 [==============================] - 0s 3ms/step - loss: 647.5399
Epoch 10: early stopping
Count Down: 8
Epoch 1/200
24/24 [==============================] - 0s 3ms/step - loss: 765.5328
Epoch 2/200
24/24 [

In [16]:
num_companies = 1  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days  # 24 time periods per day per stock
total_rows = len(wmt['2021-09-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(wmt['2021-09-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model_5 = Sequential()
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model_5.add(Dense(100, activation = 'relu', input_dim = wmt.iloc[:,1:-1].shape[1]))
model_5.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model_5.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model_5.compile(loss = 'mse', optimizer = opt)                                      # compiling model
es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

wmt_result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = wmt.iloc[i:train_rows + i, 1:] 
    test   = wmt.iloc[train_rows + i:train_rows + i + step_rows, 1:]  
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]


    
    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model_5.fit(X_train, y_train, epochs=200, batch_size=256, callbacks=[es])
    y_hat = model_5.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    wmt_result = pd.concat([wmt_result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

Epoch 1/200
24/24 [==============================] - 1s 3ms/step - loss: 19001.4570
Epoch 2/200
24/24 [==============================] - 0s 3ms/step - loss: 15277.7998
Epoch 3/200
24/24 [==============================] - 0s 3ms/step - loss: 5851.0854
Epoch 4/200
24/24 [==============================] - 0s 3ms/step - loss: 488.6411
Epoch 5/200
24/24 [==============================] - 0s 3ms/step - loss: 266.7505
Epoch 6/200
24/24 [==============================] - 0s 3ms/step - loss: 218.4875
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 199.1642
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 180.8703
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 175.8136
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 166.5312
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 165.9984
Epoch 12/200
24/24 [==============================] - 0s 3ms/step - loss: 163.8163
Epoch 13

In [17]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [18]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [19]:
new_rows = []
stock_result = aapl_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['AAPL',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,AAPL,9.373206,4.449571,5.68681,0.988342


In [20]:
new_rows = []
stock_result = f_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['F',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,F,1.775852,10.21376,0.989336,1.040944


In [21]:
new_rows = []
stock_result = nvda_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['NVDA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,NVDA,19.752165,10.960379,14.396849,1.070593


In [22]:
new_rows = []
stock_result = tsla_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['TSLA',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,TSLA,50.220057,23.459356,37.62317,1.22284


In [23]:
new_rows = []
stock_result = wmt_result
# Calculate RMSE 
rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
# Calculate MAPE % 
mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
# Calculate MPE % 
mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
# Calculate MTT in seconds 
mtt = np.mean(stock_result['TRAIN_DURATION'])
new_row = pd.Series(['WMT',rmse, mape, mpe, mtt], index=metrics_df.columns)
new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)
metrics_df

,stock_ID,RMSE,MAPE,MPE,MTT
0,WMT,6.313507,3.401291,2.216182,1.046531
